# Portée des Variables en Python : Locales, Globales, `global` et `nonlocal`

Dans cette section, nous allons explorer la **distinction entre variables locales et globales**, ainsi que l’utilisation des mots-clés **`global`** et **`nonlocal`** pour gérer les états dans les fonctions, notamment les fonctions imbriquées. 

## Qu’est-ce que la Portée ?
La **portée** (ou *scope*) d’une variable détermine où elle est accessible dans le code. Python suit la règle **LEGB** (*Local, Enclosing, Global, Built-in*) pour résoudre les noms.

Commençons par les variables locales et globales !

## Variables Locales vs Globales

### Définitions
- **Locale** : Définie dans une fonction, accessible uniquement à l’intérieur de cette fonction.
- **Globale** : Définie en dehors de toute fonction, accessible partout dans le module.

### Règles
- Une variable est locale par défaut si elle est assignée dans une fonction.
- Pour modifier une variable globale dans une fonction, il faut utiliser `global`.

### Exemple Simple
Illustrons la distinction.

In [12]:
# Variable globale
compteur = 0

def incrementer():
    """Tente d’incrémenter une variable."""
    # Variable locale par défaut (pas de référence à la globale sans `global`)
    compteur = 1  # Nouvelle variable locale, pas la globale
    print("Compteur local :", compteur)


In [13]:
print("Avant :", compteur)  

Avant : 0


In [14]:
incrementer()   

Compteur local : 1


In [15]:
 
print("Après :", compteur)  

Après : 0



- **`compteur` global** : Défini à 0 en dehors de la fonction.
- **`compteur` local** : Dans `incrementer()`, l’assignation crée une nouvelle variable locale, laissant la globale intacte.
- **Problème** : Sans `global`, la fonction ne peut pas modifier la variable globale.

Corrigeons cela avec `global` !

## Utilisation de `global`

Le mot-clé **`global`** permet à une fonction de modifier une variable globale.

### Syntaxe
```python
global nom_variable
```

### Exemple

Modifions la variable globale.

In [16]:
# Variable globale
compteur = 0

def incrementer():
    """Incrémente la variable globale compteur."""
    global compteur  # Déclare que nous utilisons la globale
    compteur += 1
    print("Compteur dans la fonction :", compteur)


In [17]:
print("Avant :", compteur)  

Avant : 0


In [18]:
incrementer()

Compteur dans la fonction : 1


In [19]:
print("Après :", compteur)


Après : 1


In [20]:
incrementer()

Compteur dans la fonction : 2


In [21]:
print("Après encore :", compteur) 

Après encore : 2



- **`global compteur`** : Indique que la fonction modifie la variable globale.
- **Résultat** : Chaque appel à `incrementer()` met à jour `compteur` globalement.
- **Attention** : L’abus de `global` peut rendre le code difficile à suivre ; préférez les retours de fonctions quand possible.

Passons aux fonctions imbriquées et `nonlocal` !

## Variables dans les Fonctions Imbriquées et `nonlocal`

Dans une fonction imbriquée, une variable peut être :
- **Locale** : Définie dans la fonction interne.
- **Enclosing** : Définie dans la fonction englobante.
- **Globale** : Définie à l’extérieur.

Le mot-clé **`nonlocal`** permet de modifier une variable de la portée englobante (*enclosing scope*).

### Exemple Simple
Créons une fonction avec une closure.

In [22]:
def externe():
    """Fonction externe avec une variable locale."""
    compteur = 0
    
    def interne():
        """Tente de modifier la variable englobante."""
        compteur = 1  # Crée une nouvelle variable locale
        print("Compteur interne :", compteur)
    
    interne()
    print("Compteur externe :", compteur)


In [23]:
externe()

Compteur interne : 1
Compteur externe : 0



- **`compteur` dans `externe`** : Variable locale à la portée englobante (0).
- **`compteur` dans `interne`** : Nouvelle variable locale (1), ne modifie pas celle de `externe`.
- **Problème** : Sans `nonlocal`, `interne` ne peut pas accéder à la variable englobante pour la modifier.

Corrigeons avec `nonlocal` !

In [24]:
def externe():
    """Fonction externe avec une variable modifiable."""
    compteur = 0
    
    def interne():
        """Modifie la variable englobante."""
        nonlocal compteur  # Référence la variable de la portée englobante
        compteur += 1
        print("Compteur interne :", compteur)
    
    interne()
    print("Compteur externe :", compteur)



In [25]:
externe()

Compteur interne : 1
Compteur externe : 1



- **`nonlocal compteur`** : Permet à `interne` de modifier la variable `compteur` de `externe`.
- **Résultat** : La modification est visible dans la portée englobante.
- **Limitation** : `nonlocal` ne fonctionne pas pour les variables globales, uniquement pour les portées englobantes.

Comparons `global` et `nonlocal` dans un exemple avancé !

## Comparaison `global` et `nonlocal`

### Exemple
Utilisons les deux dans une structure imbriquée.

In [26]:
# Variable globale
total_global = 0

def gestionnaire():
    """Fonction englobante avec une variable locale."""
    total_local = 10
    
    def incrementer_global():
        """Modifie la variable globale."""
        global total_global
        total_global += 1
        print("Total global dans incrementer_global :", total_global)
    
    def incrementer_local():
        """Modifie la variable englobante."""
        nonlocal total_local
        total_local += 5
        print("Total local dans incrementer_local :", total_local)
    
    incrementer_global()
    incrementer_local()
    print("Total local dans gestionnaire :", total_local)



In [27]:

print("Global avant :", total_global)


Global avant : 0


In [28]:
gestionnaire()

Total global dans incrementer_global : 1
Total local dans incrementer_local : 15
Total local dans gestionnaire : 15


In [29]:
print("Global après :", total_global)

Global après : 1


## Exemple Avancé : Compteur Persistant

Créons une application avec un état persistant utilisant `global` et `nonlocal`.

In [30]:
# Variable globale
total_operations = 0

def gestion_operations():
    """Gère des opérations avec un compteur local et global."""
    compteur_local = 0
    
    def enregistrer_operation():
        """Enregistre une opération aux deux niveaux."""
        global total_operations
        nonlocal compteur_local
        total_operations += 1
        compteur_local += 1
        print(f"Opération enregistrée - Local : {compteur_local}, Global : {total_operations}")
    
    def obtenir_stats():
        """Retourne les statistiques actuelles."""
        return {"local": compteur_local, "global": total_operations}
    
    return enregistrer_operation, obtenir_stats



In [31]:

# Création de deux gestionnaires
gestion1_enregistrer, gestion1_stats = gestion_operations()
gestion2_enregistrer, gestion2_stats = gestion_operations()


In [32]:
gestion1_enregistrer()


Opération enregistrée - Local : 1, Global : 1


In [33]:
gestion2_enregistrer()


Opération enregistrée - Local : 1, Global : 2


In [34]:
gestion1_enregistrer()


Opération enregistrée - Local : 2, Global : 3


In [35]:
print("Stats gestion1 :", gestion1_stats())


Stats gestion1 : {'local': 2, 'global': 3}


In [36]:
print("Stats gestion2 :", gestion2_stats())

Stats gestion2 : {'local': 1, 'global': 3}


## Conclusion

Cette section vous a permis de maîtriser :
- La **distinction entre variables locales et globales** et leurs portées.
- L’utilisation de **`global`** pour modifier des variables globales dans une fonction.
- L’utilisation de **`nonlocal`** pour gérer les variables des portées englobantes dans des fonctions imbriquées.

Ces outils sont puissants pour gérer l’état, mais utilisez-les avec parcimonie pour éviter des dépendances complexes. Expérimentez pour bien comprendre les subtilités des portées !